In [12]:
import pandas as pd
import numpy as np
from scipy import stats
import math
from sklearn import preprocessing

'''Main function'''
def main():
    df = pd.read_csv('googleplaystore.csv')      # read raw CSV
    df = df.reset_index()
    print(df.shape)
    
    i = df[df['Content Rating'].isnull()].index
    df = df.drop(i)
    i = df[df['Type'].isnull()].index
    df = df.drop(i)
    i = df[df['Type'].isnull()].index
    df = df.drop(i)
    data = df.drop('Current Ver',axis=1)
    data = data.drop('Android Ver',axis=1)
    le = preprocessing.LabelEncoder()
    data['App'] = le.fit_transform(data['App'])
    x=tuple(data['Category'].unique())
    d={}
    for i,j in enumerate(x):
        #print(i,j)
        d.update({j:(i+1)})
    #print(d)
    data['Category'] = data['Category'].map(d)
    #category_list = data['Category'].unique().tolist() 
    #category_list = ['cat_' + word for word in category_list]
    #data = pd.concat([data, pd.get_dummies(data['Category'], prefix='cat')], axis=1)
    le = preprocessing.LabelEncoder()
    data['Genres'] = le.fit_transform(data['Genres'])
    le = preprocessing.LabelEncoder()
    data['Content Rating'] = le.fit_transform(data['Content Rating'])
    data['Price'] = pd.to_numeric(data['Price'].apply(lambda x : x.strip('$')))
    data['Installs'] = pd.to_numeric(data['Installs'].apply(lambda x : x.strip('+').replace(',', '')))
    data['Type'] = pd.get_dummies(data['Type'])
    data['Size'] = data['Size'].apply(lambda x: x.strip('M').strip('k'))
    data['Last Updated']=pd.to_datetime(data['Last Updated'],infer_datetime_format=True)
    df_t = data
    df_t['Size'] = (pd.to_numeric(df_t['Size'], errors='coerce'))
    size = df_t.groupby('Category').median()['Size']
    i = df_t[df_t['Size'].isnull()].index
    for indx in i:
        df_t['Size'][indx] = size[df_t[df_t['index']==indx]['Category']]
    i = df_t[df_t['Rating'].isnull()].index
    df_t['Rating'] = (pd.to_numeric(df_t['Rating'], errors='coerce'))
    rating = df_t.groupby('Category').median()['Rating']
    for indx in i:
        df_t['Rating'][indx] = rating[df_t[df_t['index']==indx]['Category']]
    dropset = ['index']
    df_t = df_t.drop(dropset,axis=1)
    df_t['Reviews'] = pd.to_numeric(df_t['Reviews'])
    y = boxplot_outlier(df_t, 'Reviews', 'playstore_box.csv')
    #y = boxplot_outlier(data, 'Ball Possession %', 'F18_ball_possession_box.csv')
    #y = boxplot_outlier(data, 'Attempts', 'F18_attempts_box.csv')
    #y = boxplot_outlier(data, 'On-Target', 'F18_on_target_box.csv')
    #y = boxplot_outlier(data, 'Off-Target', 'F18_off_target_box.csv')
    #y = boxplot_outlier(data, 'Corners', 'F18_corners_box.csv')
    #y = boxplot_outlier(data, 'Pass Accuracy %', 'F18_pass_accuracy_box.csv')
    #y = boxplot_outlier(data, 'Passes', 'F18_passes_box.csv')
    #y = pre_process_grubbs(data, 'Goal Scored', 'F18_goal_scored_grubb.csv')
    #y = pre_process_grubbs(data, 'Ball Possession %', 'F18_ball_possession_grubb.csv')
    #y = pre_process_grubbs(data, 'Attempts', 'F18_attempts_grubb.csv')
    #y = pre_process_grubbs(data, 'On-Target', 'F18_on_target_grubb.csv')
    #y = pre_process_grubbs(data, 'Off-Target', 'F18_off_target_grubb.csv')
    #y = pre_process_grubbs(data, 'Corners', 'F18_corners_grubb.csv')
    #y = pre_process_grubbs(data, 'Pass Accuracy %', 'F18_pass_accuracy_grubb.csv')
    y = pre_process_grubbs(df_t, 'Reviews', 'playstore_grubb.csv')
    pre_process_original(df_t)
    
def pre_process_original(df):
    df.to_csv('playstore_og.csv', sep=',', index=False)

def boxplot_outlier(data, target, filename):
    Q1 = data[target].quantile(0.25)
    Q3 = data[target].quantile(0.75)
    iqr = stats.iqr(data[target])#IQR(data)
    data = data[(data[target]>(Q1-1.5*iqr)) & (data[target]<(Q3+1.5*iqr))]
    data.to_csv(filename, sep=',', index=False)
    return data

''' Grubbs-Test algorithm'''
def grubbs_test(target, N, df, a = 0.05): 
    p = 1-(a/(2*N))
    nn = N-2
    value = stats.t.ppf(p, nn)
    value**=2
    thresh = math.sqrt(value/(nn*value)) * (N-1)/math.sqrt(N)
    y = df[target]
    mean = y.mean()
    std = y.std()
    term_factor_max = 0
    term_factor_min = 0
    mean_dev = abs(y-mean)
    for i in range(N):
        y_max = mean_dev.idxmax()
        y_min = mean_dev.idxmin()
        if y_max == term_factor_max and y_min == term_factor_min:
            break
        term_factor_max = y_max
        term_factor_min = y_min
        G1 = abs(mean_dev[y_min])/std
        G2 = abs(mean_dev[y_max])/std
        if G1>thresh:
            mean_dev = mean_dev[mean_dev != mean_dev[y_min]]
            df = df[y != y[y_min]]
        if G2>thresh:
            mean_dev = mean_dev[mean_dev != mean_dev[y_max]]
            df = df[y != y[y_max]]
        #print(i, thresh, G1, y_min, G2, y_max)
    #p = stats.t.cdf(value, nn)
    return df

def pre_process_grubbs(df,target, filename):
    df = grubbs_test(target, df.shape[0], df)
    df.to_csv(filename, sep=',', index=False)

''' Main entry point'''
main()

(10841, 14)


/anaconda2/envs/jupyterlab/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda2/envs/jupyterlab/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda2/envs/jupyterlab/lib/python3.7/site-packages/ipykernel_launcher.py:112: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
